In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import acnportal
import sklearn

from copy import deepcopy
import warnings
import pytz
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from enum import Enum
from collections import namedtuple

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge

In [3]:
def ev_fleet_level_2_network(transformer_cap=30, evse_per_phase=2):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(evse_per_phase)]
    BC_ids = ['BC-{0}'.format(i) for i in range(evse_per_phase)]
    CA_ids = ['CA-{0}'.format(i) for i in range(evse_per_phase)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [4]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

In [5]:
level2_ev_fleet_network = ev_fleet_level_2_network(transformer_cap=30, evse_per_phase=2)
level2_ev_fleet_network.magnitudes

24.096385542168676
43.47826086956522


array([43.47826087, 43.47826087, 43.47826087, 24.09638554, 24.09638554,
       24.09638554])

In [6]:
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [7]:
# Events with 32 EVs per weekday
sessions_32 = get_synth_events(32)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [18]:
Schedulingimportance = namedtuple("Schedulingimportance", "importancelevel")
# evSchedulingimportance = Schedulingimportance('high')
# print(evSchedulingimportance.level)

In [19]:
session_copy = deepcopy(sessions_32)
# print(type(session_copy))
session_list = list(session_copy.queue)

def tag_specific_session_for_scheduling(session_list, session_id: str):
    new_session_list = []
    for session in session_list:
        if session[1].ev._session_id == session_id:
            modified_session = (session[0], session[1], Schedulingimportance('high'))
            new_session_list.append(modified_session)
        else:
            new_session_list.append((session[0], session[1], Schedulingimportance('low')))
    return new_session_list

new_session_list = tag_specific_session_for_scheduling(session_list, 'session_31')

for session in new_session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type, session[2].importancelevel)

645 session_17 756 8.571633345132364 Plugin low
649 session_6 792 12.110101561854618 Plugin low
646 session_23 761 5.422587850448381 Plugin low
651 session_31 729 17.440221915045356 Plugin high
660 session_19 774 17.053694121976633 Plugin low
653 session_14 776 7.137961633173925 Plugin low
655 session_5 771 11.965093467891686 Plugin low
662 session_15 777 3.8288481980606854 Plugin low
668 session_8 718 14.420706854011568 Plugin low
672 session_3 753 15.00813793994914 Plugin low
661 session_22 740 3.1298356521511517 Plugin low
657 session_11 783 5.126300686648129 Plugin low
669 session_25 737 27.8824150341117 Plugin low
664 session_27 781 13.405355824968364 Plugin low
657 session_29 773 16.127828484300814 Plugin low
672 session_1 786 23.146765010058033 Plugin low
673 session_16 798 6.2068463804823875 Plugin low
673 session_7 718 12.046540621207903 Plugin low
671 session_18 763 6.369188803495007 Plugin low
753 session_9 776 10.883646718921325 Plugin low
685 session_20 792 14.706823487616

In [9]:
def run_experiment(network, algorithm, events):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    timezone = pytz.timezone('Australia/Melbourne')

    # Start and End times are used when collecting data.
    # start = timezone.localize(datetime(2019, 6, 1))
    # end = timezone.localize(datetime(2019, 7, 1))

    start = timezone.localize(datetime(2023, 10, 3))
    end = timezone.localize(datetime(2023, 10, 4))

    sch = deepcopy(algorithm)
    cn = deepcopy(network)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = acnsim.Simulator(cn, sch, events, start, period=PERIOD, verbose=False, signals=signals)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    sim.run()

    r = {'proportion_of_energy_delivered': acnsim.proportion_of_energy_delivered(sim),
         'energy_delivered': sum(ev.energy_delivered for ev in sim.ev_history.values()),
         'num_swaps': cn.swaps,
         'num_never_charged': cn.never_charged,
         'energy_cost': acnsim.energy_cost(sim),
         'demand_charge': acnsim.demand_charge(sim)
         }
    
    # r['total_cost'] = r['energy_cost'] + r['demand_charge'] # original
    r['total_cost'] = r['energy_cost'] # modified
    r['$/kWh'] = r['total_cost'] / r['energy_delivered']
    return r

In [10]:
uncontrolled = algorithms.UncontrolledCharging()
llf = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

In [11]:
cost_min_obj = [modified_adacharge.ObjectiveComponent(modified_adacharge.total_energy, 1000),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-5),
                modified_adacharge.ObjectiveComponent(modified_adacharge.equal_share, 1e-12)
               ]
# peak limit: total aggregated current limit 
# cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)
cost_min = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=150, max_recompute=1)

In [ ]:
level2_30kW_untrl_32 = run_experiment(level2_ev_fleet_network, uncontrolled, deepcopy(sessions_32))

In [13]:
level2_30kW_llf_32 = run_experiment(level2_ev_fleet_network, llf, deepcopy(sessions_32))

Unplugged EV session_9 from station BC-1 before full charge
Will plug in priority EV session_24
Unplugged EV session_18 from station CA-0 before full charge
Will plug in priority EV session_10
Unplugged EV session_26 from station AB-1 before full charge
Will plug in priority EV session_31
Unplugged EV session_27 from station CA-1 before full charge
Will plug in priority EV session_11
Unplugged EV session_20 from station BC-1 before full charge
Will plug in priority EV session_30
Unplugged EV session_25 from station AB-1 before full charge
Will plug in priority EV session_3
Unplugged EV session_14 from station CA-1 before full charge
Will plug in priority EV session_21
Unplugged EV session_28 from station AB-1 before full charge
Will plug in priority EV session_22


In [14]:
level2_30kW_cost_min_32 = run_experiment(level2_ev_fleet_network, cost_min, deepcopy(sessions_32))

Unplugged EV session_17 from station AB-0 before full charge
Will plug in priority EV session_11


In [15]:
ev_32 = pd.DataFrame({
    'Level 2: Unctrl: 30 kW : 6 EVSEs':  level2_30kW_untrl_32,
    'Level 2: LLF: 30 kW : 6 EVSEs': level2_30kW_llf_32,
    'Level 2: Min Cost: 30 kW : 6 EVSEs': level2_30kW_cost_min_32
})

In [16]:
ev_32

,Level 2: Unctrl: 30 kW : 6 EVSEs,Level 2: LLF: 30 kW : 6 EVSEs,Level 2: Min Cost: 30 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.670271,0.509623
energy_delivered,464.029919,311.025833,236.480417
num_swaps,21.000000,19.000000,13.000000
num_never_charged,0.000000,6.000000,12.000000
energy_cost,34.594683,24.178295,17.691412
demand_charge,1023.660000,476.312100,476.312100
total_cost,34.594683,24.178295,17.691412
$/kWh,0.074553,0.077737,0.074811


In [15]:
ev_32.to_csv("results/32_EV_simulation.csv")

In [16]:
pd.read_csv("results/32_EV_simulation.csv", index_col=0)

,Level 2: Unctrl: 30 kW : 6 EVSEs,Level 2: LLF: 30 kW : 6 EVSEs,Level 2: Min Cost: 30 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.422235,0.446793
energy_delivered,399.089403,168.509583,178.310417
num_swaps,22.000000,13.000000,13.000000
num_never_charged,0.000000,13.000000,13.000000
energy_cost,22.440797,9.475294,10.026395
demand_charge,856.152000,361.538100,458.087850
total_cost,878.592797,371.013394,468.114245
$/kWh,2.201494,2.201735,2.625277


In [17]:
level2_35kw_ev_fleet_network = ev_fleet_level_2_network(transformer_cap=35, evse_per_phase=2)
level2_35kw_ev_fleet_network.magnitudes

28.112449799196785
50.724637681159415


array([50.72463768, 50.72463768, 50.72463768, 28.1124498 , 28.1124498 ,
       28.1124498 ])

In [ ]:
level2_35kW_untrl_32 = run_experiment(level2_35kw_ev_fleet_network, uncontrolled, deepcopy(sessions_32))

In [19]:
level2_35kW_llf_32 = run_experiment(level2_35kw_ev_fleet_network, llf, deepcopy(sessions_32))

Unplugged EV session_9 from station CA-1 before full charge
Will plug in priority EV session_7
Unplugged EV session_26 from station AB-1 before full charge
Will plug in priority EV session_24
Unplugged EV session_27 from station BC-1 before full charge
Will plug in priority EV session_10
Unplugged EV session_18 from station AB-0 before full charge
Will plug in priority EV session_31
Unplugged EV session_17 from station CA-0 before full charge
Will plug in priority EV session_2
Unplugged EV session_25 from station BC-1 before full charge
Will plug in priority EV session_11
Unplugged EV session_5 from station CA-0 before full charge
Will plug in priority EV session_30
Unplugged EV session_14 from station AB-0 before full charge
Will plug in priority EV session_3
Unplugged EV session_20 from station CA-1 before full charge
Will plug in priority EV session_21
Unplugged EV session_16 from station AB-1 before full charge
Will plug in priority EV session_23
Unplugged EV session_23 from statio

In [20]:
level2_35kW_cost_min_32 = run_experiment(level2_35kw_ev_fleet_network, cost_min, deepcopy(sessions_32))

Unplugged EV session_18 from station AB-0 before full charge
Will plug in priority EV session_11


In [22]:
ev_32_35kW = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 6 EVSEs':  level2_35kW_untrl_32,
    'Level 2: LLF: 35 kW : 6 EVSEs': level2_35kW_llf_32,
    'Level 2: Min Cost: 35 kW : 6 EVSEs': level2_35kW_llf_32
})

In [23]:
ev_32_35kW

,Level 2: Unctrl: 35 kW : 6 EVSEs,Level 2: LLF: 35 kW : 6 EVSEs,Level 2: Min Cost: 35 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.731308,0.731308
energy_delivered,464.029919,339.348750,339.348750
num_swaps,21.000000,19.000000,19.000000
num_never_charged,0.000000,6.000000,6.000000
energy_cost,34.594683,26.282712,26.282712
demand_charge,1023.660000,511.830000,511.830000
total_cost,34.594683,26.282712,26.282712
$/kWh,0.074553,0.077450,0.077450


In [24]:
# Events with 26 EVs per weekday
sessions_26 = get_synth_events(26)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [25]:
uncontrolled_26 = algorithms.UncontrolledCharging()
llf_26 = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

In [26]:
cost_min_obj_26 = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.total_energy, 1000),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-5),
                modified_adacharge.ObjectiveComponent(modified_adacharge.equal_share, 1e-12)
               ]
# peak limit: total aggregated current limit 
# cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)
cost_min_26 = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj_26, solver="MOSEK", quantize=True, reallocate=False, peak_limit=300, max_recompute=1)

In [27]:
level2_35kw_26_ev_fleet_network = ev_fleet_level_2_network(transformer_cap=35, evse_per_phase=2)

28.112449799196785
50.724637681159415


In [ ]:
level2_35kW_untrl_26 = run_experiment(level2_35kw_26_ev_fleet_network, uncontrolled, deepcopy(sessions_26))

In [29]:
level2_35kW_llf_26 = run_experiment(level2_35kw_26_ev_fleet_network, llf, deepcopy(sessions_26))

Unplugged EV session_18 from station AB-0 before full charge
Will plug in priority EV session_15
Unplugged EV session_8 from station CA-1 before full charge
Will plug in priority EV session_6
Unplugged EV session_4 from station AB-1 before full charge
Will plug in priority EV session_23
Unplugged EV session_17 from station AB-0 before full charge
Will plug in priority EV session_0
Unplugged EV session_6 from station CA-0 before full charge
Will plug in priority EV session_1
Unplugged EV session_5 from station AB-1 before full charge
Will plug in priority EV session_22
Unplugged EV session_24 from station AB-1 before full charge
Will plug in priority EV session_21
Unplugged EV session_22 from station BC-1 before full charge
Will plug in priority EV session_12


In [37]:
level2_35kW_cost_min_26 = run_experiment(level2_35kw_26_ev_fleet_network, cost_min_26, deepcopy(sessions_26))

Unplugged EV session_18 from station CA-0 before full charge
Will plug in priority EV session_14
Unplugged EV session_8 from station CA-1 before full charge
Will plug in priority EV session_24
Unplugged EV session_4 from station AB-1 before full charge
Will plug in priority EV session_23


In [38]:
ev_26_35kW = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 6 EVSEs':  level2_35kW_untrl_26,
    'Level 2: LLF: 35 kW : 6 EVSEs': level2_35kW_llf_26,
    'Level 2: Min Cost: 35 kW : 6 EVSEs': level2_35kW_cost_min_26
})

In [39]:
ev_26_35kW

,Level 2: Unctrl: 35 kW : 6 EVSEs,Level 2: LLF: 35 kW : 6 EVSEs,Level 2: Min Cost: 35 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.610762,0.484815
energy_delivered,304.478464,185.963750,147.615833
num_swaps,15.000000,12.000000,12.000000
num_never_charged,0.000000,5.000000,7.000000
energy_cost,23.138953,14.002298,10.573219
demand_charge,1023.660000,505.315800,511.830000
total_cost,23.138953,14.002298,10.573219
$/kWh,0.075995,0.075296,0.071627


In [40]:
cost_min_obj_26_2 = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.total_energy, 1000),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-5),
                modified_adacharge.ObjectiveComponent(modified_adacharge.equal_share, 1e-11)
               ]
# peak limit: total aggregated current limit 
# cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)
cost_min_26_2 = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj_26_2, solver="MOSEK", quantize=True, reallocate=True, peak_limit=300, max_recompute=1)

In [41]:
level2_35kW_cost_min_26_2 = run_experiment(level2_35kw_26_ev_fleet_network, cost_min_26_2, deepcopy(sessions_26))

Unplugged EV session_18 from station AB-1 before full charge
Will plug in priority EV session_14
Unplugged EV session_4 from station AB-0 before full charge
Will plug in priority EV session_23
Unplugged EV session_6 from station AB-1 before full charge
Will plug in priority EV session_0


In [42]:
ev_26_35kW = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 6 EVSEs':  level2_35kW_untrl_26,
    'Level 2: LLF: 35 kW : 6 EVSEs': level2_35kW_llf_26,
    'Level 2: Min Cost: 35 kW : 6 EVSEs': level2_35kW_cost_min_26_2
})

In [44]:
ev_26_35kW

,Level 2: Unctrl: 35 kW : 6 EVSEs,Level 2: LLF: 35 kW : 6 EVSEs,Level 2: Min Cost: 35 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.610762,0.486013
energy_delivered,304.478464,185.963750,147.980590
num_swaps,15.000000,12.000000,12.000000
num_never_charged,0.000000,5.000000,7.000000
energy_cost,23.138953,14.002298,10.609457
demand_charge,1023.660000,505.315800,543.935700
total_cost,23.138953,14.002298,10.609457
$/kWh,0.075995,0.075296,0.071695


### Simulation (work in progress)
- Create a separate code module for MPC based Adaptive charging algorithm (done)
- Modify queue mechanism 
- Modify ada code to simulate fulfillment of charging requirement for high priority EVs
- Add new optimization objective 